# The code used for new task

## This code is to create a structure as similiar as yolo

## PLEASE CHANGE THE PATH ACCORDINGLY

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Create the Directory Structure:

In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Define paths
dir_path = "/content/drive/MyDrive/Object Detection"
filepath = os.path.join(dir_path, "task-data/NewTask")

labels_source = os.path.join(filepath, "task-new-xml")
images_source = os.path.join(filepath, "captured_frames")
dataset_path = os.path.join(filepath, 'dataset-new')  # New structured dataset location

output_path = os.path.join(dataset_path, "newYoloLabel")
classes_path = os.path.join(dataset_path, "classes.txt")



Conversi XML into TXT

In [7]:
import os
import xml.etree.ElementTree as ET
from pathlib import Path

def convert_voc_to_yolo(xml_path, output_dir, classes):
    """
    Convert a single VOC XML file to YOLO format
    """
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Get image dimensions
        size = root.find('size')
        if size is None:
            print(f"Warning: No size element found in {xml_path}")
            return False

        width_elem = size.find('width')
        height_elem = size.find('height')

        if width_elem is None or height_elem is None:
            print(f"Warning: Missing width or height in {xml_path}")
            return False

        width = int(width_elem.text)
        height = int(height_elem.text)

        # Prepare output file
        txt_filename = Path(xml_path).stem + '.txt'
        txt_path = os.path.join(output_dir, txt_filename)

        objects_found = 0
        with open(txt_path, 'w') as f:
            for obj in root.findall('object'):
                # Get class name
                name_elem = obj.find('name')
                if name_elem is None:
                    print(f"Warning: Object without name in {xml_path}")
                    continue

                class_name = name_elem.text

                # Check if class is in our list
                if class_name not in classes:
                    print(f"Warning: Class '{class_name}' not in classes list, skipping...")
                    continue

                class_id = classes.index(class_name)

                # Get bounding box coordinates
                bndbox = obj.find('bndbox')
                if bndbox is None:
                    print(f"Warning: Object without bndbox in {xml_path}")
                    continue

                xmin_elem = bndbox.find('xmin')
                ymin_elem = bndbox.find('ymin')
                xmax_elem = bndbox.find('xmax')
                ymax_elem = bndbox.find('ymax')

                if None in [xmin_elem, ymin_elem, xmax_elem, ymax_elem]:
                    print(f"Warning: Incomplete bndbox in {xml_path}")
                    continue

                xmin = int(xmin_elem.text)
                ymin = int(ymin_elem.text)
                xmax = int(xmax_elem.text)
                ymax = int(ymax_elem.text)

                # Convert to YOLO format
                x_center = (xmin + xmax) / 2 / width
                y_center = (ymin + ymax) / 2 / height
                w = (xmax - xmin) / width
                h = (ymax - ymin) / height

                # Ensure values are within [0,1] range
                x_center = max(0, min(1, x_center))
                y_center = max(0, min(1, y_center))
                w = max(0, min(1, w))
                h = max(0, min(1, h))

                # Write to file
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {w:.6f} {h:.6f}\n")
                objects_found += 1

        print(f"Converted {xml_path.name}: {objects_found} objects")
        return objects_found > 0

    except Exception as e:
        print(f"Error processing {xml_path}: {str(e)}")
        return False

def convert_dataset(voc_annotations_dir, output_dir, classes_file):
    """
    Convert all VOC XML files in a directory to YOLO format
    """
    # Check if directories exist
    if not os.path.exists(voc_annotations_dir):
        print(f"Error: VOC annotations directory '{voc_annotations_dir}' does not exist")
        return

    if not os.path.exists(classes_file):
        print(f"Error: Classes file '{classes_file}' does not exist")
        return

    # Read class names
    with open(classes_file, 'r') as f:
        classes = [line.strip() for line in f.readlines() if line.strip()]

    print(f"Classes loaded: {classes}")

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Find XML files
    xml_files = list(Path(voc_annotations_dir).glob('*.xml'))
    print(f"Found {len(xml_files)} XML files")

    if len(xml_files) == 0:
        print("No XML files found. Check the directory path.")
        return

    # Convert each XML file
    successful_conversions = 0
    for xml_file in xml_files:
        if convert_voc_to_yolo(xml_file, output_dir, classes):
            successful_conversions += 1

    print(f"Conversion completed: {successful_conversions}/{len(xml_files)} files converted successfully")

if __name__ == "__main__":
    # Your paths
    convert_dataset(
        voc_annotations_dir=labels_source,
        output_dir=output_path,
        classes_file=classes_path
    )

Classes loaded: ['incomplete', 'completed']
Found 303 XML files
Converted frame_20250917_134002_563012.xml: 1 objects
Converted frame_20250917_134118_609832.xml: 1 objects
Converted frame_20250917_134106_613336.xml: 1 objects
Converted frame_20250917_132833_895817.xml: 1 objects
Converted frame_20250917_134125_410471.xml: 1 objects
Converted frame_20250917_134129_620386.xml: 1 objects
Converted frame_20250917_150033_365906.xml: 1 objects
Converted frame_20250917_134002_043839.xml: 1 objects
Converted frame_20250917_134014_638261.xml: 1 objects
Converted frame_20250917_134019_345642.xml: 1 objects
Converted frame_20250917_132817_184752.xml: 1 objects
Converted frame_20250917_132911_896999.xml: 1 objects
Converted frame_20250917_132841_722953.xml: 2 objects
Converted frame_20250917_134123_313235.xml: 1 objects
Converted frame_20250917_153002_945785.xml: 1 objects
Converted frame_20250917_132919_243027.xml: 1 objects
Converted frame_20250917_134120_180070.xml: 1 objects
Converted frame_20

Verify the classes.txt

In [8]:
# Run this separately to check your classes file
classes_file = classes_path
with open(classes_file, 'r') as f:
    classes = [line.strip() for line in f.readlines() if line.strip()]
print("Classes in file:", classes)

Classes in file: ['incomplete', 'completed']
